In [54]:
# function untuk clear display
from IPython.display import clear_output

In [42]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [43]:
import json
with open("Dataset/recipes.json", "r") as file:
    data = json.load(file)

In [44]:
# untuk melihat jumlah resep di dataset dan melihat sample resep
print(f"Total recipes: {len(data)}")
print("Sample Recipe:", data[0])

Total recipes: 49
Sample Recipe: {'recipe_name': 'Nasi Goreng', 'main_component': 'Nasi', 'ingredients': ['1 porsi nasi putih', '2 siung bawang merah', '2 siung bawang putih', 'kecap manis secukupnya', 'garam secukupnya', 'merica secukupnya', '100 gr ayam suwir', '1 butir telur', '1 batang daun bawang', 'minyak goreng secukupnya'], 'instructions': ['1.  Panaskan minyak, tumis bawang merah dan bawang putih hingga harum.', '2. Masukkan ayam suwir dan tumis hingga matang.', '3. Tambahkan nasi putih dan aduk rata.', '4. Bumbui dengan kecap manis, garam, dan merica.', '5. Goreng telur dan sajikan di atas nasi goreng.', '6. Taburi dengan daun bawang.'], 'prep_time': '10 minutes', 'cook_time': '15 minutes', 'servings': 2}


In [45]:
# untuk mencari nama resep yang exact
def find_recipe(food_name):
  for recipe in data:
    if recipe["recipe_name"].lower() == food_name.lower():
      return recipe
  return None

In [46]:
# mendisplay detail resep makanan
def display_recipe(recipe):
    if not recipe:
        return

    print(f"Resep {recipe['recipe_name']}")
    print("\nBahan:")
    for ingredient in recipe["ingredients"]:
        print(f"- {ingredient}")

    print("\nCara Pembuatan:")
    for step, instruction in enumerate(recipe["instructions"], 1):
        print(f"{instruction}")

In [47]:
from fuzzywuzzy import process

In [74]:
def find_closest_recipe(food_name):
    recipe_names = [recipe["recipe_name"] for recipe in data]
    match, score = process.extractOne(food_name, recipe_names)
    if score > 86:
        return next(recipe for recipe in data if recipe["recipe_name"] == match)
    return None

In [49]:
# untuk ngematch input user dengan main component di dataset
def find_closest_comp(comp_name):
    all_components = []
    for recipe in data:
        components = recipe.get("main_component")
        if isinstance(components, list):
            all_components.extend([c.lower() for c in components])
        elif isinstance(components, str):
            all_components.append(components.lower())
    match, score = process.extractOne(comp_name.lower(), all_components)
    if score > 70:
        return match # return nama main component yg cocok
    return None

In [50]:
# mendisplay list makanan dengan main component yang diinput user
def list_food_by_main_component(main_component):
    if not main_component:
        return []

    main_component = main_component.lower()
    results = []
    for recipe in data:
        components = recipe.get("main_component")
        if isinstance(components, list):
            if any(main_component == c.lower() for c in components):
                results.append(recipe)
        elif isinstance(components, str):
            if main_component == components.lower():
                results.append(recipe)
    return results

In [84]:
# function untuk looping
def search_again():
    clear_output(wait=True)
    opt = input("\nApakah anda ingin mencari resep makanan lain? (ya/tidak)\n")
    if opt.lower() == 'tidak':
        return False
    return True

In [79]:
def main():
    while True:
        clear_output(wait=True)
        print("=== AI Resep Makanan ===")
        print("1. Cari berdasarkan nama makanan")
        print("2. Cari berdasarkan komponen utama makanan")
        print("3. Keluar")

        try:
            print('\nPilih opsi (1/2/3): ')
            choice = int(input().strip())
              
            if choice == 1:
                clear_output(wait=True)
                print('Masukkan nama makanan yang ingin anda cari: ')
                food_name = str(input())
                recipe = find_recipe(food_name)
                clear_output(wait=True)

                if not recipe:
                    recipe = find_closest_recipe(food_name)
                    if not recipe:
                        print(f"Maaf, tidak ada resep dengan nama '{food_name}'.\n")
                    else:
                        print(f"Apakah resep makanan ini yang anda cari?\n")

                display_recipe(recipe)

            elif choice == 2:
                clear_output(wait=True)
                print('\nMasukkan komponen utama yang ingin dicari: ')
                main_component = input()
                closest_component = find_closest_comp(main_component)


                if not closest_component:
                    print(f"Maaf, tidak ada resep yang mengandung '{main_component}'.\n")
                    continue

                recipes = list_food_by_main_component(closest_component)

                if not recipes:
                    print(f"Maaf, tidak ada resep dengan komponen utama '{main_component}'.\n")
                else:
                    print(f"Ditemukan {len(recipes)} resep dengan komponen utama '{closest_component}':\n")
                    for idx, recipe in enumerate(recipes, 1):
                        print(f"{idx}. {recipe['recipe_name']}")
                    while True:
                        try:
                            sub_choice = int(input("\nPilih nomor resep untuk melihat detailnya (0 untuk batal): ").strip())
                            if sub_choice == 0:
                                 print("Membatalkan pilihan.")
                                 break

                            if 1 <= sub_choice <= len(recipes):
                                selected_recipe = recipes[sub_choice - 1]
                                clear_output(wait=True)
                                display_recipe(selected_recipe)
                                break
                            else:
                                 print("Nomor resep tidak valid, silakan masukkan angka yang sesuai.")
                        except ValueError:
                            print("Input tidak valid, silakan masukkan angka.")

            elif choice == 3:
                clear_output(wait=True)
                print("** Exitting Program... **")
                break

            else:
                clear_output(wait=True)
                print("Opsi tidak valid. Silakan coba lagi.")

        except ValueError:
            clear_output(wait=True)
            print("Input tidak valid. Silakan masukkan angka.")

        if not search_again():
            clear_output(wait=True)
            print("** Exitting Program... **")
            break

In [85]:
main()

** Exitting Program... **
